In [5]:
from sklearn.decomposition import NMF
import numpy as np
import sys
import os
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stopwords = ENGLISH_STOP_WORDS

In [6]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
newsgroups_train = fetch_20newsgroups()
newsgroups_trainlabel = newsgroups_train.target
vectorizer = TfidfVectorizer(stop_words=stopwords)
vectorstrain = vectorizer.fit_transform(newsgroups_train.data)

In [7]:
import elasticsearch
es = elasticsearch.Elasticsearch()
def show_top20(classifier, vectorizer,no_top_words,type1,data):
    
    index_no = type1 + data
    doc20Ngtype = type1 + data + "type"
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(classifier.components_):
        print(i)
        top_words=category.argsort()[:-no_top_words - 1:-1]
        words = []
        for j in top_words:
            print(feature_names[j],category[j])
            words.append(feature_names[j])
            
        es.index(index=index_no, doc_type=doc20Ngtype, body={
                'doc_id': i,
                'doc_text': ":".join(words)
                })
        
        print("**************************************************************************")

In [8]:
nmf = NMF(n_components=20, init='random', random_state=0).fit(vectorstrain)

In [9]:
no_top_words = 20
show_top20(nmf, vectorizer, no_top_words,"nmf","20ng")

0
people 1.4756082255762746
gun 1.205469221158711
don 1.1385755195928113
think 0.9428867083097036
like 0.8893337197548258
just 0.8710343885808988
car 0.760548047759888
guns 0.6964789853363197
good 0.6845313404119557
time 0.6452247378975209
right 0.6330380129733512
know 0.6182014503773037
make 0.6088601003353089
things 0.5226293122211071
way 0.5185333319366353
say 0.5165851663492519
going 0.5101680184142614
government 0.5019229953841854
ve 0.4858784419318435
want 0.4834833151136502
**************************************************************************
1
windows 1.7164463281323097
window 0.8703849604061925
file 0.8002688667205365
dos 0.7636369366251392
files 0.5942791541948089
program 0.48832726667404536
ms 0.43583232560002405
use 0.3863178201608571
server 0.31440822106050226
using 0.3123336628658578
mouse 0.30245481220531223
problem 0.3002506666717715
version 0.2906164053315226
help 0.27915720577667547
running 0.2775452108773661
screen 0.273446833058436
application 0.270745893861483

In [10]:
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(vectorstrain)

In [11]:
no_top_words = 20
show_top20(lda, vectorizer, no_top_words,"lda","20ng")

0
tittle 1.0290417472753248
pallis 0.696450004914516
dimitrios 0.39804054323373206
alexandre 0.23686411693854265
dumas 0.23685143275994117
92716 0.23676175271628336
ucivax 0.23664083001909675
4188 0.23660303941132446
cltittle 0.23654426747802687
1600x1200x256 0.2266832766174188
blitz 0.21932682739776338
katrina 0.21817285107907722
gowan 0.21808608003038624
004253agrgb 0.2180764633408383
cocker 0.218062471205769
agrgb 0.21800482955381137
2bd73621 0.14381559668073124
imagens 0.14342672663977868
3894 0.14307861333332303
haston 0.10143628694283977
**************************************************************************
1
zmed16 2.5258343363650564
aario 1.0728653839818723
a137490 1.0728293130901063
zircon 0.9047788448777372
_defined_ 0.24555387989628927
fostex 0.18999997995760567
191100 0.1676559007752805
16094 0.16744778316620365
bigoting 0.16597338203166526
alesis 0.10902004912135388
sfnet 0.1011951192453459
eros 0.10098400723946262
013034 0.10086585662432386
27070 0.1008407918279995
73

In [12]:
dirPath= "/Users/panktibhalani/Downloads/DUC2001"
document_map={}
for fname in os.listdir(dirPath):
    f_path= dirPath + "/" + fname
    if (not os.path.isdir(f_path)) and (not fname.startswith('notes')) and (not fname.startswith('annotations')):
        with open(f_path,'r', encoding="utf8", errors='ignore') as file:
            document_map[fname.lower()]=file.read()

In [13]:
vectorizerdcu = TfidfVectorizer(stop_words=stopwords)
vectorsdcu = vectorizerdcu.fit_transform(document_map)

In [14]:
nmfdcu = NMF(n_components=20, init='random', random_state=0).fit(vectorsdcu)


In [15]:
no_top_words = 20
show_top20(nmfdcu, vectorizerdcu, no_top_words,"nmf","duc")

0
sjmn91 0.47689486241715906
06283083 0.059411116269226474
06276078 0.059411116269226474
06084228 0.059411116269226474
06012224 0.059411116269226474
06191081 0.059411116269226474
06169114 0.059411116269226474
06182091 0.059411116269226474
06301029 0.059411116269226474
06290146 0.059411116269226474
06312120 0.059411116269226474
06193235 0.059411116269226474
06212161 0.059411116269226474
06246065 0.059411116269226474
06187248 0.059411116269226474
06193081 0.04912413654365777
0076 3.285978706038008e-41
la071589 1.7944693334590185e-41
la093089 1.7645592726125035e-41
0006 5.315983569384464e-46
**************************************************************************
1
0079 0.40400942015855934
ap880914 0.2776221639852056
0027 0.1543728125808689
ap891210 0.13638564292385685
wsj880621 0.13638564292385685
ap881211 0.051465891136291735
ap890111 5.989209427993258e-09
0227 5.989209427993258e-09
la021689 2.6034341045776668e-09
0217 2.603434104577666e-09
ap880927 8.910300746907363e-11
0089 6.202714

15
ft934 1.353617082170949
8748 0.22526594065438776
10911 0.22526594065438776
11014 0.22526594065438776
12800 0.22526594065438776
13350 0.22526594065438776
5781 0.22526594065438776
8628 0.22526594065438776
9116 0.22526594065438776
0036 2.2777299080926443e-22
ap900323 8.576059225247221e-23
ap901010 8.576059225247221e-23
wsj920211 8.576059225247221e-23
0039 2.461727872180936e-24
la041889 9.268844367041089e-25
la081890 9.268844367041089e-25
ap900607 9.268844367041089e-25
0017 8.338169944872802e-27
ap880928 5.559436537463186e-27
0054 5.559436537463186e-27
**************************************************************************
16
0163 0.9329060837539083
wsj880923 0.3512557743322015
la030889 0.3512557743322015
la120290 0.3512557743322015
0036 0.029517935882033122
ap900323 0.011114029167020516
ap901010 0.011114029167020516
wsj920211 0.011114029167020516
0008 2.003714488519942e-09
0100 1.3309418904735986e-09
wsj911030 1.0739078007055604e-09
ap890803 1.0739078007055604e-09
ap890930 7.1332961

In [16]:
ldadcu = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(vectorsdcu)
no_top_words = 20
show_top20(ldadcu, vectorizerdcu, no_top_words,"lda","duc")

0
0024 1.6630619756923788
0126 1.6251374088142803
wsj910628 1.408245621818105
wsj911121 1.404258583673341
wsj880617 1.3672277316163766
0109 1.3262388627660031
0136 1.2418537511158392
la093089 1.1835836074792048
ap890326 0.9158317430374787
0174 0.8953597942132084
ap880419 0.870902748825521
341 0.8661209674580904
ap890802 0.8603566486086314
la012590 0.8592863786956725
0135 0.857485995718436
la030489 0.8560161505243725
ap890325 0.8515033604630374
0105 0.841755645330341
0068 0.8390595643881977
ap900306 0.8353894051280244
**************************************************************************
1
ft923 2.8676660532204257
0115 1.9086824668125122
0109 1.630381403106848
0090 1.6302875080070869
wsj911106 1.4063964183027824
wsj870123 1.3774764047858628
wsj900705 1.3730754480492686
wsj910709 1.369651524130603
0145 1.348239304424484
wsj910326 1.347010056822049
0101 1.3214534451419682
ap881222 1.1763256694580413
0089 1.1100112244873328
ap880705 1.100878268560319
0051 0.9723592590442024
la081489 0.